In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as ny

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID", axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
indep=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

In [11]:
dep=dataset['Purchased']

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep, dep, test_size=1/3, random_state=0)

In [13]:
indep.shape

(400, 3)

In [14]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [17]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

param_grid={'criterion':['gini', 'entropy'], 'splitter':['best', 'random'], 'max_features':['auto','sqrt','log2']}

grid=GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, cv=5, n_jobs=-1, scoring='f1_weighted')
#to tune a model
grid.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\ProgramData\anaconda3\Lib\site-packages\

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [18]:
#to know the best tune parameter
re = grid.cv_results_
#print(re)

grid_prediction=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)
print(cm)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88        85
           1       0.78      0.82      0.80        49

    accuracy                           0.85       134
   macro avg       0.84      0.84      0.84       134
weighted avg       0.85      0.85      0.85       134

[[74 11]
 [ 9 40]]


In [19]:
#to know the f1 score
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_prediction, average='weighted')
print("The f1_macro value of best parameter is {}:".format(grid.best_params_),f1_macro)

The f1_macro value of best parameter is {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'random'}: 0.8513503909026298


In [20]:
#as a addition to know a receiver operator characteristic area under curve value to identify best classification
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(x_test)[:,-1])

0.8434573829531813

In [21]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002304,0.000764,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.002888,0.003867,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.005029,0.001744,0.005430,0.001647,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.852030,0.813179,0.887907,0.867097,0.849295,0.027019,5
3,0.004209,0.001422,0.007253,0.001715,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.843811,0.828959,0.850543,0.886792,0.844370,0.850895,0.019301,3
4,0.003700,0.000247,0.005297,0.001485,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.804764,0.831253,0.796284,0.886792,0.867097,0.837238,0.034971,7
5,0.003146,0.001642,0.006997,0.002839,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.849794,0.852030,0.793565,0.905069,0.943041,0.868700,0.051244,1
6,0.000762,0.000491,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000311,0.000417,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.002632,0.000772,0.004968,0.001920,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.813511,0.814409,0.924528,0.823140,0.840370,0.042365,6
9,0.002201,0.001269,0.007198,0.002152,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.868752,0.813179,0.847020,0.887907,0.856867,0.025389,2


In [22]:
age_input=float(input('Age'))
EstimatedSalary_input=float(input('EstimatedSalary'))
Gender_Male_input=float(input('Gender_Male'))

Age 26
EstimatedSalary 80000
Gender_Male 1


In [23]:
grid.predict([[age_input, EstimatedSalary_input, Gender_Male_input]])

array([1], dtype=int64)